In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Alles

In [ ]:
alles = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Alles/data.h5")
utils.peek(alles, "build/drosophila/Alles")
alles.obs.head()

In [ ]:
alles.obs["cluster"] = "cluster_" + alles.obs["cluster"].astype(int).astype(str)
alles.obs.dtypes

In [ ]:
alles_model = cb.directi.fit_DIRECTi(
    alles, alles.uns["seurat_genes"],
    **fixed_model_kwargs, h_dim=512
)
alles.latent = alles_model.inference(alles)

In [ ]:
ax = alles.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/drosophila/Alles/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alles.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/drosophila/Alles/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = alles.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/drosophila/Alles/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
alles.write_dataset("build/drosophila/Alles/Alles.h5")

In [ ]:
%%capture capio
alles_models = [alles_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    alles_models.append(cb.directi.fit_DIRECTi(
        alles, alles.uns["seurat_genes"],
        **fixed_model_kwargs, h_dim=512,
        random_seed=i
    ))
alles_blast = cb.blast.BLAST(alles_models, alles)
alles_blast.save("build/drosophila/Alles")

In [ ]:
with open("build/drosophila/Alles/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/drosophila/Alles/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(alles_blast, "build/drosophila/Alles")

In [ ]:
%%writefile build/drosophila/Alles/predictable.txt
cell_ontology_class
cell_type1
cluster